In [1]:
import numpy as np
import os
import pandas as pd
import time
from itertools import product
import math

In [2]:
#### disease similarities 합치는 과정
#disease_ph = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_mimminerc.tsv", index_col=0)
#disease_ge = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_genetic.tsv")
#disease_ph2 = pd.DataFrame()
#for i in disease_ph.columns.values:
#    for j in disease_ph.index.values:
#        temp_df = pd.DataFrame(data = [[int(i), int(j), disease_ph[[i]].loc[j].values[0]]], columns = ['Disease_ID1', 'Disease_ID2', 'Similarity_Score'] )
#        disease_ph2 = disease_ph2.append(temp_df)
#df_disease_sim = pd.merge(left = disease_ph2, right= disease_ge, left_on = ['Disease_ID1', 'Disease_ID2'], right_on = ['Disease_ID1', 'Disease_ID2'])
#df_disease_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_whole.tsv", sep='\t', index=False)
#df_disease_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_whole.tsv", sep='\t', index=False)
#df_disease_sim = df_disease_sim.rename(columns = {'Similarity_Score_x':'PheSim', 'Similarity_Score_y':'GeSim'})
df_disease_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_whole.tsv")

In [3]:
#### drug similarities 합치는 과정
#drug_ch = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_structure.tsv")
#drug_ph = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_phenotype.tsv")
#df_drug_sim = pd.merge(left = drug_ch, right= drug_ph, left_on = ['DrugBank_ID1', 'DrugBank_ID2'], right_on = ['DrugBank_ID1', 'DrugBank_ID2'])
#df_drug_sim = df_drug_sim.rename(columns = {'Similarity_Score_x' : 'CheSim', 'Similarity_Score_y':'PheSim'})
#df_drug_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_whole.tsv", index=False, sep='\t')
#df_drug_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_whole.tsv", index=False, sep='\t')
df_drug_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_whole.tsv")

In [4]:
#### PREDICT용 새로운 association
#asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/9_gold_learning_association.tsv")
#new_asso = asso[asso['drug'].isin(df_drug_sim['DrugBank_ID1'].drop_duplicates())]
#new_asso.index = range(len(new_asso))
#new_asso.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/10_PREDICT_association.tsv", sep='\t', index=False)
#new_asso.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/10_PREDICT_association.tsv", sep='\t', index=False)
new_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/10_PREDICT_association.tsv")

In [5]:
print len(df_drug_sim)
df_drug_sim.head(1)

207936


,DrugBank_ID1,DrugBank_ID2,CheSim,PheSim
0,DB00215,DB00215,1.0,1.0


In [6]:
print len(df_disease_sim)
df_disease_sim.head(1)

55225


,Disease_ID1,Disease_ID2,PheSim,GeSim
0,102300,102300,1.0,1.0


In [7]:
print len(new_asso)
new_asso.head(1)

107160


,drug,disease,association
0,DB00190,102300,1


In [8]:
pos = new_asso[new_asso.association == 1]
neg = new_asso[new_asso.association == 0]
print len(pos)
print len(neg)

1397
105763


In [93]:
pos.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/test_pos.tsv", sep='\t', index=False)
neg.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/test_neg.tsv", sep='\t', index=False)

In [9]:
new_asso_li = list()
for index, row in new_asso.iterrows():
    query_drug = row['drug']
    query_disease = row['disease']
    query_asso = row['association']
    new_asso_li.append((query_drug, query_disease, query_asso))

In [10]:
new_pos_li = list()
for index2, row2 in pos.iterrows():
    pos_drug = row2['drug']
    pos_disease = row2['disease']
    new_pos_li.append((pos_drug, pos_disease)) 

In [11]:
print 'start : ' + time.strftime('%a %H:%M:%S')
desc_pair = list(product(new_asso_li, new_pos_li))
print 'end : ' + time.strftime('%a %H:%M:%S')

start : Mon 22:45:42
end : Mon 22:45:55


In [33]:
print len(desc_pair)
desc_pair[1395:1399]

149702520


[(('DB00190', 102300, 1), ('DB00868', 608088)),
 (('DB00190', 102300, 1), ('DB00775', 608320)),
 (('DB00190', 102400, 0), ('DB00190', 102300)),
 (('DB00190', 102400, 0), ('DB00190', 125320))]

In [43]:
descriptor = pd.DataFrame()
cnt = 0
print 'start : ' + time.strftime('%a %H:%M:%S')
for pair in desc_pair:
    query_drug = pair[0][0]
    query_disease = pair[0][1]
    query_asso = pair[0][2]
    pos_drug = pair[1][0]
    pos_disease = pair[1][1]
    if (query_drug == pos_drug) & (query_disease == pos_disease):
            continue
    else:
        found_drug = df_drug_sim[(df_drug_sim.DrugBank_ID1 == query_drug) & (df_drug_sim.DrugBank_ID2 == pos_drug)]
        drug_cheSim = found_drug['CheSim'].values[0]
        drug_pheSim = found_drug['PheSim'].values[0]
    
        found_disease = df_disease_sim[(df_disease_sim.Disease_ID1 == query_disease) & (df_disease_sim.Disease_ID2 == pos_disease)]
        disease_pheSim = found_disease['PheSim'].values[0]
        disease_geSim = found_disease['GeSim'].values[0]
    
        drug_sim = [drug_cheSim, drug_pheSim]
        disease_sim = [disease_geSim, disease_pheSim]
        
        sim = [math.sqrt(x[0]*x[1]) for x in list(product(drug_sim, disease_sim))]
        temp_df = pd.DataFrame(data = [[query_drug, query_disease, query_asso, sim[0], sim[1], sim[2], sim[3]]],
                           columns=['drug', 'disease', 'association', 'DrChe_DiGe', 'DrChe_DiPhe', 'DrPhe_DiGe', 'DrPhe_DiPhe'])
        descriptor = descriptor.append(temp_df)
        cnt = cnt +1
        if cnt % 1000 == 0:
            print "{} : ".format(cnt) + time.strftime('%a %H:%M:%S')
print 'end : ' + time.strftime('%a %H:%M:%S')

start : Mon 23:10:07
1000 : Mon 23:10:24
2000 : Mon 23:10:43
3000 : Mon 23:11:01
4000 : Mon 23:11:20
5000 : Mon 23:11:38
6000 : Mon 23:11:56
7000 : Mon 23:12:14
8000 : Mon 23:12:32
9000 : Mon 23:12:49
10000 : Mon 23:13:07
11000 : Mon 23:13:26
12000 : Mon 23:13:43
13000 : Mon 23:14:01
14000 : Mon 23:14:19
15000 : Mon 23:14:38
16000 : Mon 23:14:56
17000 : Mon 23:15:14
18000 : Mon 23:15:31
19000 : Mon 23:15:48
20000 : Mon 23:16:06
21000 : Mon 23:16:24
22000 : Mon 23:16:42
23000 : Mon 23:16:59
24000 : Mon 23:17:15
25000 : Mon 23:17:30
26000 : Mon 23:17:45
27000 : Mon 23:17:59
28000 : Mon 23:18:14
29000 : Mon 23:18:28
30000 : Mon 23:18:43
31000 : Mon 23:18:58
32000 : Mon 23:19:12
33000 : Mon 23:19:27
34000 : Mon 23:19:42
35000 : Mon 23:19:56
36000 : Mon 23:20:11
37000 : Mon 23:20:26
38000 : Mon 23:20:40
39000 : Mon 23:20:55
40000 : Mon 23:21:10
41000 : Mon 23:21:25
42000 : Mon 23:21:39
43000 : Mon 23:21:54
44000 : Mon 23:22:09
45000 : Mon 23:22:24
46000 : Mon 23:22:39
47000 : Mon 23:22:54
4

KeyboardInterrupt: 

In [44]:
cnt

129824

In [46]:
descriptor.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/pre_desc_0_129824.tsv", sep='\t', index=False)

In [47]:
descriptor.head()

,drug,disease,association,DrChe_DiGe,DrChe_DiPhe,DrPhe_DiGe,DrPhe_DiPhe
0,DB00190,102300,1,0.970745,0.410585,0.970745,0.410585
0,DB00190,102300,1,0.982234,0.522589,0.982234,0.522589
0,DB00190,102300,1,0.982234,0.396621,0.982234,0.396621
0,DB00190,102300,1,0.617611,0.617611,0.917272,0.917272
0,DB00190,102300,1,0.607798,0.136358,0.902698,0.202518


In [64]:
print len(descriptor[(descriptor.drug == 'DB00190') & (descriptor.disease == 102300)]) # known
print len(descriptor[(descriptor.drug == 'DB00190') & (descriptor.disease == 102400)]) # unknwon
print len(descriptor[(descriptor.drug == 'DB00190') & (descriptor.disease == 173200)]) # unknwon 되다만거

1396
1397
1304


In [87]:
real_desc = descriptor.groupby(['drug','disease']).max()
#real_desc.loc['DB00190',102300] # data 접근시
## 밑에거 잘라내기. partial slicing
real_desc = real_desc.loc[('DB00190', 102300):('DB00190', 171300)] # partial slicing (range로)

In [90]:
real_desc.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/real_desc_0_129824.tsv", sep='\t')
real_desc.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/real_desc_0_129824.tsv", sep='\t')

In [119]:
##### similarity tensor product 전까지만. 
descriptor = pd.DataFrame()
cnt = 0
print 'start : ' + time.strftime('%a %H:%M:%S')
bef_descriptor = list()
for index, row in new_asso.iterrows(): # all association
    print time.strftime('%a %H:%M:%S')
    query_drug = row['drug']
    query_disease = row['disease']
    query_asso = row['association']
    similarities = list()
    for index2, row2, in pos.iterrows(): # known association
        pos_drug = row2['drug']
        pos_disease = row2['disease']
        if (query_drug == pos_drug) & (query_disease == pos_disease):
            continue
        else:
            drug_cheSim = drug_ch[(drug_ch.DrugBank_ID1 == query_drug) & (drug_ch.DrugBank_ID2 == pos_drug)]['Similarity_Score']
            drug_pheSim = drug_ph[(drug_ph.DrugBank_ID1 == query_drug) & (drug_ph.DrugBank_ID2 == pos_drug)]['Similarity_Score']
            disease_pheSim = disease_ph[[str(query_disease)]].loc[pos_disease]
            disease_geSim = disease_ge[(disease_ge.Disease_ID1 == query_disease) & (disease_ge.Disease_ID2 == pos_disease)]['Similarity_Score']
            drug_sim = [drug_cheSim.values[0], drug_pheSim.values[0]]
            disease_sim = [disease_geSim.values[0], disease_pheSim.values[0]]
            similarities.append([drug_sim, disease_sim])
            #similarities.append([math.sqrt(x[0]*x[1]) for x in list(product(drug_sim, disease_sim))])
    bef_descriptor.append(similarities)    
    cnt = cnt + 1
    print time.strftime('%a %H:%M:%S')
    if cnt % 1000 == 0 :
        print "cnt : {} ".format(cnt) + time.strftime('%a %H:%M:%S')
print 'end : ' + time.strftime('%a %H:%M:%S')

start : Mon 21:27:27
Mon 21:27:27
Mon 21:28:07
Mon 21:28:07
Mon 21:28:47
Mon 21:28:47
Mon 21:29:28
Mon 21:29:28
Mon 21:30:08
Mon 21:30:08
Mon 21:30:48
Mon 21:30:48
Mon 21:31:29
Mon 21:31:29
Mon 21:32:09
Mon 21:32:09


KeyboardInterrupt: 

In [111]:
print query_disease
print pos_disease
print disease_ph[[str(query_disease)]].loc[pos_disease]

102300
608320
102300    0.0
Name: 608320, dtype: float64


In [99]:
##### similarity tensor product 후 score까지만. 
cnt = 0
print time.strftime('%a %H:%M:%S')
bef_descriptor = list()
for index, row in new_asso.iterrows(): # all association
    query_drug = row['drug']
    query_disease = row['disease']
    query_asso = row['association']
    similarities = list()
    print time.strftime('%a %H:%M:%S')
    for index2, row2, in pos.iterrows(): # known association
        pos_drug = row2['drug']
        pos_disease = row2['disease']
        if (query_drug == pos_drug) & (query_disease == pos_disease):
            continue
        else:
            drug_cheSim = drug_ch[(drug_ch.DrugBank_ID1 == query_drug) & (drug_ch.DrugBank_ID2 == pos_drug)]['Similarity_Score']
            drug_pheSim = drug_ph[(drug_ph.DrugBank_ID1 == query_drug) & (drug_ph.DrugBank_ID2 == pos_drug)]['Similarity_Score']
            disease_pheSim = disease_ph[[str(query_disease)]].loc[pos_disease]
            disease_geSim = disease_ge[(disease_ge.Disease_ID1 == query_disease) & (disease_ge.Disease_ID2 == pos_disease)]['Similarity_Score']
            drug_sim = [drug_cheSim.values[0], drug_pheSim.values[0]]
            disease_sim = [disease_geSim.values[0], disease_pheSim.values[0]]
            similarities.append([math.sqrt(x[0]*x[1]) for x in list(product(drug_sim, disease_sim))])

    bef_descriptor.append(similarities)
    print time.strftime('%a %H:%M:%S')
    break
    cnt = cnt + 1
    if cnt % 1000 == 0 :
        print "cnt : {} ".format(cnt) + time.strftime('%a %H:%M:%S')
print time.strftime('%a %H:%M:%S')

Mon 19:48:26
Mon 19:48:26
Mon 19:49:02
Mon 19:49:02


In [98]:
cnt

0